## Imports

In [10]:
import pandas as pd
import geopandas as gpd
import googlemaps
import json
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

## Data read in

In [2]:
df = pd.read_csv("DOGE_Velocity_Tracker.csv")

In [3]:
df = df[['Main Agency', 'Location', 'Sq Ft', 'Annual Lease', 'Saved']]

In [4]:
df['Annual Lease'] = df['Annual Lease'].str.replace('$','').replace(',','',regex=True).astype(int)
df['Saved'] = df['Saved'].str.replace('$','').replace(',','',regex=True).astype(int)

In [5]:
df.columns

Index(['Main Agency', 'Location', 'Sq Ft', 'Annual Lease', 'Saved'], dtype='object')

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lon = g[0]["geometry"]["location"]["lng"]
        return (lat, lon)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Location'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [11]:
def df_to_geojson(df):
    geojson = {"type": "FeatureCollection", "features": []}
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "Main Agency": row["Main Agency"],
                "Location": row["Location"],
                "Sq Ft": row["Sq Ft"],
                "Annual Lease": row["Annual Lease"],
                "Saved": row["Saved"],
            },
            "geometry": {
                "type": "Point",
                "coordinates": [row["lon"], row["lat"]]
            }
        }
        geojson["features"].append(feature)
    return geojson

geojson_data = df_to_geojson(df)

# Save GeoJSON to a file for use in your web map
with open("data.geojson", "w") as f:
    json.dump(geojson_data, f)

In [13]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/DOGE_Velocity_Tracker
